# LimberJack.jl Tutorial

## Usage

In [1]:
# The statistical inference frame-work we will use
using Turing
using AdvancedHMC
using LogDensityProblems
using LogDensityProblemsAD
using DynamicPPL
using ForwardDiff
# Our code
using LimberJack
# Some data management libs.
using CSV
using NPZ
using YAML
#Plotting
using Plots
# Some Lin. Alg.
using LinearAlgebra
# Allows us to use Python when needed
using PythonCall
sacc = pyimport("sacc")

Python module: <module 'sacc' from '/home/jaimerz/.julia/environments/v1.7/.CondaPkg/env/lib/python3.10/site-packages/sacc/__init__.py'>

### Load data

The purpose of this notebook is show the user how to interface ```LimberJack.jl``` with ```Turing.jl``` to perform statistical inference.
The first hurdle is converting our data into a format that ```LimberJack.jl``` can parse. 
The main obstacle is the lack of a ```Julia``` implementation of ```sacc.py``` (```yaml.jl``` exists!).
Thankfully we can call ```sacc.py``` from ```Julia```. 
This allows us to load both the relevant ```.yml``` and ```.fits``` files.
These files can then me passed to  ```LimberJack.jl ```'s  ```make_data ``` function to generate the  ```meta ``` structure and the  ```files ``` dictionary composed respectively of:

    + ```meta```
        + ```tracers```: a list of all tracers 
        + ```pairs```: a list of all pairs of tracers for the $C_l$'s
        + ```cls```: the concatenated flat data vector
        + ```idx```: indecies of the data vector corresponding to the first $l$ of each $C_l$ such that ```cli = cls[idx[i]:idx[i+1]]```
        + ```cov```: covariance matrix of the data vector
        + ```inv_cov```: inverse of the former
    + ```files.npz```
        + ```ls_tracer_tracer```: an array containing the $l$'s for the tracer pair
        + ```nz_tracer```: the redshift distribution of galaxies of a given tracer as ```[z, dndz]```. 


In [2]:
sacc_path = "/home/jaimerz/PhD/LimberJack.jl/data/FD/cls_FD_covG.fits"
yaml_path = "/home/jaimerz/PhD/LimberJack.jl/data/DESY1/gcgc_gcwl_wlwl.yml"
sacc_file = sacc.Sacc().load_fits(sacc_path)
yaml_file = YAML.load_file(yaml_path)
meta, files = make_data(sacc_file, yaml_file)

data_vector = meta.data
cov_tot = meta.cov;

DESgc__0 DESgc__0 5
DESgc__1 DESgc__1 8
DESgc__2 DESgc__2 10
DESgc__3 DESgc__3 11
DESgc__4 DESgc__4 13
DESgc__0 DESwl__0 5
DESgc__0 DESwl__1 5
DESgc__0 DESwl__2 5
DESgc__0 DESwl__3 5
DESgc__1 DESwl__0 8
DESgc__1 DESwl__1 8
DESgc__1 DESwl__2 8
DESgc__1 DESwl__3 8
DESgc__2 DESwl__0 10
DESgc__2 DESwl__1 10
DESgc__2 DESwl__2 10
DESgc__2 DESwl__3 10
DESgc__3 DESwl__0 11
DESgc__3 DESwl__1 11
DESgc__3 DESwl__2 11
DESgc__3 DESwl__3 11
DESgc__4 DESwl__0 13
DESgc__4 DESwl__1 13
DESgc__4 DESwl__2 13
DESgc__4 DESwl__3 13
DESwl__0 DESwl__0 24
DESwl__0 DESwl__1 24
DESwl__0 DESwl__2 24
DESwl__0 DESwl__3 24
DESwl__1 DESwl__1 24
DESwl__1 DESwl__2 24
DESwl__1 DESwl__3 24
DESwl__2 DESwl__2 24
DESwl__2 DESwl__3 24
DESwl__3 DESwl__3 24


### Defining the model

Now it is time to create the ```Turin.jl``` model. This is done through a function using the ```@model``` decorator.
The important detail here is that the model can only be a function of the data (upon which we will condition the model to obtain constraints). The model function must accomplish the following:
1. Define the priors for the parameters. This is done using the syntax of ```Distributions.jl``` for statistical programming. Note that the nuisance parameters must follow a strict naming convention in terms of the tracer they belong to. Otherwise, they will not be recognized. Also in order to pass the nuisance parameters our machinary they must be written as a dictionary. 
2. Initiate an instance of ```cosmo = LimberJack.Cosmology``` which will generate the matter power spectrum for the sampled cosmological parameters upon initializing. 
3. Evaluate ```LimberJack.Theory(cosmo, tracers, pairs, idx, files; Nuisances=nuisances)``` which for a given instance of  ```LimberJack.Cosmology``` it will return the concatenated $C_l$'s vector for the considered tracers, tracers' pairs, their indeces in the data vector and their auxialiry files  ($l$'s and $n(z)$'s).

In [3]:
@model function model(data; #data dependence!!!
                      files=files,
                      meta=meta,
                      cov=cov_tot) 
    # Define priors
    #KiDS priors
    Ωm ~ Uniform(0.2, 0.6)
    Ωb ~ Uniform(0.028, 0.065)
    h ~ TruncatedNormal(72, 5, 0.64, 0.82)
    s8 ~ Uniform(0.4, 1.2)
    ns ~ Uniform(0.84, 1.1)

    DESgc__0_b ~ Uniform(0.8, 3.0)
    DESgc__1_b ~ Uniform(0.8, 3.0)
    DESgc__2_b ~ Uniform(0.8, 3.0)
    DESgc__3_b ~ Uniform(0.8, 3.0)
    DESgc__4_b ~ Uniform(0.8, 3.0)
    DESgc__0_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__1_dz ~ TruncatedNormal(0.0, 0.007, -0.2, 0.2)
    DESgc__2_dz ~ TruncatedNormal(0.0, 0.006, -0.2, 0.2)
    DESgc__3_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESgc__4_dz ~ TruncatedNormal(0.0, 0.01, -0.2, 0.2)
    DESwl__0_dz ~ TruncatedNormal(-0.001, 0.016, -0.2, 0.2)
    DESwl__1_dz ~ TruncatedNormal(-0.019, 0.013, -0.2, 0.2)
    DESwl__2_dz ~ TruncatedNormal(0.009, 0.011, -0.2, 0.2)
    DESwl__3_dz ~ TruncatedNormal(-0.018, 0.022, -0.2, 0.2)
    DESwl__0_m ~ Normal(0.012, 0.023)
    DESwl__1_m ~ Normal(0.012, 0.023)
    DESwl__2_m ~ Normal(0.012, 0.023)
    DESwl__3_m ~ Normal(0.012, 0.023)
    A_IA ~ Uniform(-5, 5) 
    alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("DESgc__0_b" => DESgc__0_b,
                     "DESgc__1_b" => DESgc__1_b,
                     "DESgc__2_b" => DESgc__2_b,
                     "DESgc__3_b" => DESgc__3_b,
                     "DESgc__4_b" => DESgc__4_b,
                     "DESgc__0_dz" => DESgc__0_dz,
                     "DESgc__1_dz" => DESgc__1_dz,
                     "DESgc__2_dz" => DESgc__2_dz,
                     "DESgc__3_dz" => DESgc__3_dz,
                     "DESgc__4_dz" => DESgc__4_dz,

                     "A_IA" => A_IA,
                     "alpha_IA" => alpha_IA,

                     "DESwl__0_dz" => DESwl__0_dz,
                     "DESwl__1_dz" => DESwl__1_dz,
                     "DESwl__2_dz" => DESwl__2_dz,
                     "DESwl__3_dz" => DESwl__3_dz,
                     "DESwl__0_m" => DESwl__0_m,
                     "DESwl__1_m" => DESwl__1_m,
                     "DESwl__2_m" => DESwl__2_m,
                     "DESwl__3_m" => DESwl__3_m)

    cosmology = Cosmology(Ωm, Ωb, h, ns, s8,
                          tk_mode="EisHu",
                          Pk_mode="Halofit")

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    data ~ MvNormal(theory, cov)
end;

Now it is time to condition the model on the data. This is simply done as:

In [4]:
iterations = 250
adaptation = 1000
TAP = 0.65
stat_model = model(data_vector)
alg = Turing.NUTS(adaptation, TAP)

Turing.Inference.NUTS{Turing.Essential.ForwardDiffAD{0}, (), DiagEuclideanMetric}(1000, 0.65, 10, 1000.0, 0.0)

In [5]:
using Optim

function Xi2(params; model=stat_model)
    Ωm, Ωb, h, s8,  ns,
    DESgc__0_b, DESgc__1_b, DESgc__2_b, DESgc__3_b, DESgc__4_b,
    DESgc__0_dz, DESgc__1_dz, DESgc__2_dz, DESgc__3_dz, DESgc__4_dz,
    DESwl__0_dz, DESwl__1_dz, DESwl__2_dz, DESwl__3_dz,
    DESwl__0_m, DESwl__1_m, DESwl__2_m, DESwl__3_m,
    A_IA, alpha_IA = params
    return loglikelihood(model,
                        (Ωm=Ωm, Ωb=Ωb, h=h, s8=s8, ns=ns,
                         DESgc__0_b=DESgc__0_b, DESgc__1_b=DESgc__1_b, DESgc__2_b=DESgc__2_b, DESgc__3_b=DESgc__3_b, DESgc__4_b=DESgc__4_b,
                         DESgc__0_dz=DESgc__0_dz, DESgc__1_dz=DESgc__1_dz, DESgc__2_dz=DESgc__2_dz, DESgc__3_dz=DESgc__3_dz, DESgc__4_dz=DESgc__4_dz,
                         DESwl__0_dz=DESwl__0_dz, DESwl__1_dz=DESwl__1_dz, DESwl__2_dz=DESwl__2_dz, DESwl__3_dz=DESwl__3_dz,
                         DESwl__0_m=DESwl__0_m, DESwl__1_m=DESwl__1_m, DESwl__2_m=DESwl__2_m, DESwl__3_m=DESwl__3_m,
                         A_IA=A_IA, alpha_IA=alpha_IA))
end;
    
function maximum_a_posteriori(model, lower_bound, upper_bound)
    start_value = (upper_bound .+ lower_bound) ./ 2 
    opt = optimize((v)->-Xi2(v), lower_bound, upper_bound, start_value, Fminbox())
    return Optim.minimizer(opt)
end

maximum_a_posteriori (generic function with 1 method)

In [6]:
lowers = [0.2, 0.028, 0.64, 0.4, 0.84, 
          0.8, 0.8, 0.8, 0.8, 0.8, 
          -0.2, -0.2, -0.2, -0.2, -0.2,
          -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2,
          -5.0, -5.0]
uppers = [0.6, 0.065, 0.82, 1.2, 1.1, 
          3.0, 3.0, 3.0, 3.0, 3.0, 
          0.2, 0.2, 0.2, 0.2, 0.2,
          0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,
          5.0, 5.0];

In [7]:
#MAP = maximum_a_posteriori(stat_model, lowers, uppers)

# MAP from a preivous run
MAP = [ 2.89781002e-01,  4.77686400e-02,  7.38011259e-01,  7.98425957e-01,
        9.69728709e-01,  1.49721568e+00,  1.81865817e+00,  1.79001308e+00,
        2.18263403e+00,  2.24598151e+00, -3.46696435e-03, -3.77762873e-03,
       -1.60856663e-03, -2.09706269e-03,  5.92820210e-04, -1.05679892e-02,
        5.08483360e-03,  4.75152724e-03, -8.16263963e-04,  1.90524810e-02,
        3.83250992e-03,  2.35920245e-02, -1.54272168e-03,  3.16431800e-01,
       -2.64120253e-01]

pars_names = ["Ωm", "Ωb", "h", "s8",  "ns",
              "DESgc__0_b", "DESgc__1_b","DESgc__2_b","DESgc__3_b","DESgc__4_b",
              "DESgc__0_dz", "DESgc__1_dz", "DESgc__2_dz", "DESgc__3_dz", "DESgc__4_dz",
              "DESwl__0_dz", "DESwl__1_dz", "DESwl__2_dz", "DESwl__3_dz",
              "DESwl__0_m", "DESwl__1_m", "DESwl__2_m", "DESwl__3_m",
              "A_IA", "alpha_IA",];

In [8]:
# Get the Hessian
hess = ForwardDiff.hessian(Xi2, MAP)
inv_hess = inv(hess);

In [9]:
# Turn the Hessian into more of a covariance Matrix
w, v = eigen(inv_hess)
hess_cov = v * (diagm(abs.(w)) * v')
hess_cov = tril(hess_cov) + triu(hess_cov', 1)
hess_cov = Hermitian(hess_cov)
hess_cov = convert(Matrix{Float64}, hess_cov)

25×25 Matrix{Float64}:
  0.00174914    0.000350097  -0.000612938  …  -0.0028114    -0.00302873
  0.000350097   0.000298532   0.000347349      0.000466794  -0.0156898
 -0.000612938   0.000347349   0.0022484        0.00558196   -0.0643927
 -0.000469425   0.000353485   0.00195119       0.00700356   -0.127265
 -0.00240725    0.000819839   0.00385055       0.00988849   -0.0810492
  0.00182176   -0.00427704   -0.0193218    …  -0.0462215     0.782665
  0.000338195  -0.000452606  -0.00322188      -0.0145882     0.28996
  9.78142e-6   -0.00140936   -0.00663414      -0.021336      0.395109
  0.000267924  -0.00152566   -0.00761907      -0.0251625     0.454262
  0.000351658   0.000142816  -0.000537036     -0.00887275    0.22432
 -0.000381022   0.00128569    0.0054013    …   0.0113454    -0.179698
 -4.8538e-6     4.58442e-5    0.000276293      0.00122037   -0.0136195
  0.000212337   0.00130788    0.00510216       0.0121463    -0.175984
 -6.21897e-6    0.000755145   0.00312955       0.00749001   -0.

### Sampling

In [10]:
spl = Sampler(alg, stat_model)

Sampler{Turing.Inference.NUTS{Turing.Essential.ForwardDiffAD{0}, (), DiagEuclideanMetric}}(Turing.Inference.NUTS{Turing.Essential.ForwardDiffAD{0}, (), DiagEuclideanMetric}(1000, 0.65, 10, 1000.0, 0.0), DynamicPPL.Selector(0x00001d58304a19fa, :default, false))

In [11]:
context = stat_model.context
varinfo = DynamicPPL.VarInfo(stat_model, context)

TypedVarInfo{NamedTuple{(:Ωm, :Ωb, :h, :s8, :ns, :DESgc__0_b, :DESgc__1_b, :DESgc__2_b, :DESgc__3_b, :DESgc__4_b, :DESgc__0_dz, :DESgc__1_dz, :DESgc__2_dz, :DESgc__3_dz, :DESgc__4_dz, :DESwl__0_dz, :DESwl__1_dz, :DESwl__2_dz, :DESwl__3_dz, :DESwl__0_m, :DESwl__1_m, :DESwl__2_m, :DESwl__3_m, :A_IA, :alpha_IA), Tuple{DynamicPPL.Metadata{Dict{VarName{:Ωm, Setfield.IdentityLens}, Int64}, Vector{Uniform{Float64}}, Vector{VarName{:Ωm, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, DynamicPPL.Metadata{Dict{VarName{:Ωb, Setfield.IdentityLens}, Int64}, Vector{Uniform{Float64}}, Vector{VarName{:Ωb, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, DynamicPPL.Metadata{Dict{VarName{:h, Setfield.IdentityLens}, Int64}, Vector{Truncated{Normal{Float64}, Continuous, Float64}}, Vector{VarName{:h, Setfield.IdentityLens}}, Vector{Float64}, Vector{Set{DynamicPPL.Selector}}}, DynamicPPL.Metadata{Dict{VarName{:s8, Setfield.IdentityLens}, Int64}, Vecto

In [12]:
ℓ = LogDensityProblemsAD.ADgradient(DynamicPPL.LogDensityFunction(varinfo, stat_model, context))
lπ = Base.Fix1(LogDensityProblems.logdensity, ℓ)
∂lπ∂θ(x) = LogDensityProblems.logdensity_and_gradient(ℓ, x)
metric = DenseEuclideanMetric(hess_cov)
hamiltonian = AdvancedHMC.Hamiltonian(metric, lπ, ∂lπ∂θ)

Hamiltonian(metric=DenseEuclideanMetric(diag=[0.0017491365706465293, 0.0 ...]), kinetic=GaussianKinetic())

In [ ]:
# Set the number of samples to draw and warmup iterations
n_samples, n_adapts = 2_000, 1_000
initial_ϵ = find_good_stepsize(hamiltonian, MAP)
integrator = Leapfrog(initial_ϵ)

# Define an HMC sampler, with the following components
#   - multinomial sampling scheme,
#   - generalised No-U-Turn criteria, and
#   - windowed adaption for step-size and diagonal mass matrix
proposal = AdvancedHMC.NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator)
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

# Run the sampler to draw samples from the specified Gaussian, where
#   - `samples` will store the samples
#   - `stats` will store diagnostic statistics for each sample
samples, stats = sample(hamiltonian, proposal, MAP, n_samples, adaptor, n_adapts; progress=true)

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, true, false, true)
└ @ AdvancedHMC ~/.julia/packages/AdvancedHMC/eLdgX/src/hamiltonian.jl:49
Sampling   0%|                               |  ETA: 18:42:41
  iterations:                    2
  n_steps:                       1
  is_accept:                     true
  acceptance_rate:               0.0
  log_density:                   8622.109007042482
  hamiltonian_energy:            -8602.949569277567
  hamiltonian_energy_error:      0.0
  max_hamiltonian_energy_error:  Inf
  tree_depth:                    0
  numerical_error:               true
  step_size:                     0.3253429190059853
  nom_step_size:                 0.3253429190059853
  is_adapt:                      true
  mass_matrix:                   DenseEuclideanMetric(diag=[0.0017491365706465293, 0.0 ...])